In [ ]:
# Import
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
# Loading the dataframe
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None, names=['target', 'id', 'date', 'flag', 'user', 'text'])
# Map target values to sentiment labels to match pre-trained model (-1, 0, 1)
df['sentiment'] = df['target'].apply(lambda x: -1 if x == 0 else (1 if x == 4 else 0))

In [ ]:
# Split dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)